In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from copy import copy

# **Get DataFrames**

In [ ]:
def getSizedDataFramesBest(size):
    men_list1 = menBest(size).keys()
    men_list = []
    for man in men_list1:
      men_list.append( 'm'+str(man))

    women_list1 = womenBest(size, menBest(size)).keys()
    women_list = []
    for woman in women_list1:
      women_list.append('w'+str(woman))
    
    women_df = pd.DataFrame( womenBest(size, menBest(size)))
    women_df.index = men_list
    women_df.columns = women_list
    men_df = pd.DataFrame(menBest(size))
    men_df.index = men_list
    men_df.columns = women_list
    return women_list, men_list, women_df, men_df    

In [ ]:
  def getSizedDataFramesWorst(size):
    men_list1 = menWorst(size).keys()
    men_list = []
    for man in men_list1:
      men_list.append( 'm'+str(man))

    women_list1 = womenWorst(size).keys()
    women_list = []
    for woman in women_list1:
      women_list.append('w'+str(woman))

    women_df = pd.DataFrame( womenWorst(size) )
    women_df.index = men_list
    women_df.columns = women_list
    men_df = pd.DataFrame(menWorst(size))
    men_df.index = men_list
    men_df.columns = women_list
    return women_list, men_list, women_df, men_df

# **Gale-Shapley Algorithm**

In [ ]:
#def gale_shapley(women_list, men_list, men_df={}, women_df={}):
def gale_shapley( lista ):
  women_list = lista[0]
  men_list = lista[1]
  men_df = lista[2]
  women_df = lista[3]
  # dict to control which women each man can make proposals
  women_available = {man:women_list for man in men_list}
  # waiting list of men that were able to create pair on each iteration
  waiting_list = []
  # dict to store created pairs
  proposals = {}
  # variable to count number of iterations
  count = 0
  # while not all men have pairs
  while len(waiting_list)<len(men_list):
      # man makes proposals
      for man in men_list:
          if man not in waiting_list:
              # each man make proposal to the top women from it's list
              women = women_available[man]
              best_choice = men_df.loc[man][men_df.loc[man].index.isin(women)].idxmin()
              proposals[(man, best_choice)]=(men_df.loc[man][best_choice],
                                                  women_df.loc[man][best_choice])
      # if women have more than one proposals 
      # she will choose the best option
      overlays = Counter([key[1] for key in proposals.keys()])
      # cycle to choose the best options
      for women in overlays.keys():
          if overlays[women]>1:
              # pairs to drop from proposals
              pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() 
                      if women in pair}.items(), 
                    key=lambda x: x[1][1]
                    )[1:]
              # if man was rejected by woman
              # there is no pint for him to make proposal 
              # second time to the same woman
              for p_to_drop in pairs_to_drop:
                  del proposals[p_to_drop[0]]
                  _women = copy(women_available[p_to_drop[0][0]])
                  _women.remove(p_to_drop[0][1])
                  women_available[p_to_drop[0][0]] = _women
      # man who successfully created pairs must be added to the waiting list 
      waiting_list = [man[0] for man in proposals.keys()]
      # update counter
      count+=1
  return proposals, count

# **Best Case Scenario**

In [ ]:
def menBest(n):
  dicMenBest={}
  for i in range(n):
    #clave='m'+str(i+1)
    clave=i+1
    valor=[]
    for j in range(n):
      if i+j+1>n:
        v=i+j+1-n
        valor.append(v)
      else:
        v=i+j+1
        valor.append(v)
    npValor= np.array(valor)
    dicMenBest[str(clave)]=npValor
  return dicMenBest

menBest(8)

{'1': array([1, 2, 3, 4, 5, 6, 7, 8]),
 '2': array([2, 3, 4, 5, 6, 7, 8, 1]),
 '3': array([3, 4, 5, 6, 7, 8, 1, 2]),
 '4': array([4, 5, 6, 7, 8, 1, 2, 3]),
 '5': array([5, 6, 7, 8, 1, 2, 3, 4]),
 '6': array([6, 7, 8, 1, 2, 3, 4, 5]),
 '7': array([7, 8, 1, 2, 3, 4, 5, 6]),
 '8': array([8, 1, 2, 3, 4, 5, 6, 7])}

In [ ]:
def womenBest(n, dic={}):
  elements=dic.items()
  dicWomenBest={}
  for i in range(n): #þ #para i = 0 
    clave=i+1 #clave = w1
    #aux=[['0'] * i for i in range(n)] # aux = []
    aux = []
    for j in range(n):
      aux.append(0)
    valor=np.array(aux)   # valor = array([], dtype=float64)
    for key, value in elements:
      #print(value)
      #print(str(clave))
      #print(np.where(value == clave))
      l=np.where(value == clave)[0][0] # key = m1 el índice de w1 en el arreglo de m1 es 0 entonces l=0
      valor[l]=key
    dicWomenBest[str(clave)]=valor
  return dicWomenBest
mB=menBest(8)
womenBest(8,mB)

{'1': array([1, 8, 7, 6, 5, 4, 3, 2]),
 '2': array([2, 1, 8, 7, 6, 5, 4, 3]),
 '3': array([3, 2, 1, 8, 7, 6, 5, 4]),
 '4': array([4, 3, 2, 1, 8, 7, 6, 5]),
 '5': array([5, 4, 3, 2, 1, 8, 7, 6]),
 '6': array([6, 5, 4, 3, 2, 1, 8, 7]),
 '7': array([7, 6, 5, 4, 3, 2, 1, 8]),
 '8': array([8, 7, 6, 5, 4, 3, 2, 1])}

# **Worst Case Scenario**

In [ ]:
def menWorst(n):
  dicMenWorst={} #Crear diccionario
  for i in range(n): 
    clave=str(i+1)
    valor=[]
    for j in range(n):
      if i+j+1>n:
        v=i+j+1-n
        valor.append(v)
      else:
        v=i+j+1
        valor.append(v)
    npValor= np.array(valor)
    dicMenWorst[clave]=npValor
  return dicMenWorst
menWorst(4)

{'1': array([1, 2, 3, 4]),
 '2': array([2, 3, 4, 1]),
 '3': array([3, 4, 1, 2]),
 '4': array([4, 1, 2, 3])}

In [ ]:
def womenWorst(n):
  dicWomenWorst={}
  for i in range(n):
    clave=i+1
    valor=[]
    for j in range(n):
      if i+j+1>n:
        v=i+j+2-n
        valor.append(v)
      else:
        v=i+j+2
        if v == n+1:
          v=1
        valor.append(v)
    npValor= np.array(valor)
    dicWomenWorst[str(clave)]=npValor
  return dicWomenWorst

womenWorst(4)

{'1': array([2, 3, 4, 1]),
 '2': array([3, 4, 1, 2]),
 '3': array([4, 1, 2, 3]),
 '4': array([1, 2, 3, 4])}

# **Execution**

In [ ]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [ ]:
sizes = [4, 40]
for i in sizes:
  print("Best case for size {0}".format(i))
  b = getSizedDataFramesBest(i)
  g = gale_shapley( b )
  print("Proposals:")
  print(pretty(g[0]))
  print('Number of rounds: {0}'.format( g[1]))
  print("\n")


Best case for size 4
Proposals:
('m1', 'w1')
	(1, 1)
('m2', 'w2')
	(1, 3)
('m3', 'w3')
	(1, 1)
('m4', 'w4')
	(1, 3)
None
Number of rounds: 1


Best case for size 40
Proposals:
('m1', 'w1')
	(1, 1)
('m2', 'w2')
	(1, 3)
('m3', 'w3')
	(1, 5)
('m4', 'w4')
	(1, 7)
('m5', 'w5')
	(1, 9)
('m6', 'w6')
	(1, 11)
('m7', 'w7')
	(1, 13)
('m8', 'w8')
	(1, 15)
('m9', 'w9')
	(1, 17)
('m10', 'w10')
	(1, 19)
('m11', 'w11')
	(1, 21)
('m12', 'w12')
	(1, 23)
('m13', 'w13')
	(1, 25)
('m14', 'w14')
	(1, 27)
('m15', 'w15')
	(1, 29)
('m16', 'w16')
	(1, 31)
('m17', 'w17')
	(1, 33)
('m18', 'w18')
	(1, 35)
('m19', 'w19')
	(1, 37)
('m20', 'w20')
	(1, 39)
('m21', 'w21')
	(1, 1)
('m22', 'w22')
	(1, 3)
('m23', 'w23')
	(1, 5)
('m24', 'w24')
	(1, 7)
('m25', 'w25')
	(1, 9)
('m26', 'w26')
	(1, 11)
('m27', 'w27')
	(1, 13)
('m28', 'w28')
	(1, 15)
('m29', 'w29')
	(1, 17)
('m30', 'w30')
	(1, 19)
('m31', 'w31')
	(1, 21)
('m32', 'w32')
	(1, 23)
('m33', 'w33')
	(1, 25)
('m34', 'w34')
	(1, 27)
('m35', 'w35')
	(1, 29)
('m36', 'w36

In [ ]:
sizes = [4, 40]
for i in sizes:
  print("Worst case for size {0}".format(i))
  b = getSizedDataFramesWorst(i)
  g = gale_shapley( b )
  print("Proposals:")
  print(pretty(g[0]))
  print('Number of rounds: {0}'.format( g[1]))
  print("\n")

Worst case for size 4
Proposals:
('m1', 'w4')
	(1, 4)
('m2', 'w3')
	(1, 4)
('m3', 'w2')
	(1, 4)
('m4', 'w1')
	(1, 4)
None
Number of rounds: 1


Worst case for size 40
Proposals:
('m1', 'w40')
	(1, 40)
('m2', 'w39')
	(1, 40)
('m3', 'w38')
	(1, 40)
('m4', 'w37')
	(1, 40)
('m5', 'w36')
	(1, 40)
('m6', 'w35')
	(1, 40)
('m7', 'w34')
	(1, 40)
('m8', 'w33')
	(1, 40)
('m9', 'w32')
	(1, 40)
('m10', 'w31')
	(1, 40)
('m11', 'w30')
	(1, 40)
('m12', 'w29')
	(1, 40)
('m13', 'w28')
	(1, 40)
('m14', 'w27')
	(1, 40)
('m15', 'w26')
	(1, 40)
('m16', 'w25')
	(1, 40)
('m17', 'w24')
	(1, 40)
('m18', 'w23')
	(1, 40)
('m19', 'w22')
	(1, 40)
('m20', 'w21')
	(1, 40)
('m21', 'w20')
	(1, 40)
('m22', 'w19')
	(1, 40)
('m23', 'w18')
	(1, 40)
('m24', 'w17')
	(1, 40)
('m25', 'w16')
	(1, 40)
('m26', 'w15')
	(1, 40)
('m27', 'w14')
	(1, 40)
('m28', 'w13')
	(1, 40)
('m29', 'w12')
	(1, 40)
('m30', 'w11')
	(1, 40)
('m31', 'w10')
	(1, 40)
('m32', 'w9')
	(1, 40)
('m33', 'w8')
	(1, 40)
('m34', 'w7')
	(1, 40)
('m35', 'w6')
	(1,